In [1]:
import rasa_core
from rasa_core.agent import Agent
from bokeh.io import output_notebook
agent = Agent.load("models/dialogue_embed")
# agent = Agent.load("models/dialogue_embed_before_norm_loss")
output_notebook()

/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


INFO:tensorflow:Restoring parameters from models/dialogue_embed/policy_0_EmbeddingPolicy/tensorflow_embedding.ckpt


/anaconda/lib/python3.6/site-packages/pykwalify/core.py:99: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  data = yaml.load(stream)


Loading BokehJS ...

In [2]:
policy = agent.policy_ensemble.policies[0]
interpreter = agent.interpreter

In [3]:
domain = agent.domain


In [4]:
from rasa_core.training.structures import StoryGraph
from rasa_core.training.dsl import StoryFileReader
from rasa_core.trackers import DialogueStateTracker
from rasa_core.channels import UserMessage
import numpy as np

def embedding_from_tracker(story, bot=False):
    reader = StoryFileReader(domain, interpreter)
    story = story.split('\n')
    story_steps = reader.process_lines(story)
#     story_graph = StoryGraph(story_steps)
    tracker = DialogueStateTracker(
            1,
            domain.slots
    )
    tracker._reset()
    for step in story_steps:
        events = step.explicit_events(domain)
        for event in events:
            tracker.update(event)
    data_X = policy.featurizer.create_X([tracker], domain)

    X, slots, prev_act = policy._create_X_slots_prev_acts(data_X)
    all_Y_d = policy._create_all_Y_d(X.shape[1])
    all_Y_d_x = np.stack([all_Y_d for _ in range(X.shape[0])])
    dial_vec = policy.session.run(policy.dial_embed, feed_dict={policy.a_in: X,
                                           policy.b_in: all_Y_d_x,
                                           policy.c_in: slots,
                                           policy.b_prev_in: prev_act})
    bot_vec = policy.session.run(policy.bot_embed, feed_dict={policy.a_in: X,
                                           policy.b_in: all_Y_d_x,
                                           policy.c_in: slots,
                                           policy.b_prev_in: prev_act})

    no_skip_gate = policy.session.run(policy.no_skip_gate, feed_dict={policy.a_in: X,
                                           policy.b_in: all_Y_d_x,
                                           policy.c_in: slots,
                                           policy.b_prev_in: prev_act})
    attn_embed = policy.session.run(policy.attn_embed, feed_dict={policy.a_in: X,
                                           policy.b_in: all_Y_d_x,
                                           policy.c_in: slots,
                                           policy.b_prev_in: prev_act})
    rnn_embed = policy.session.run(policy.rnn_embed, feed_dict={policy.a_in: X,
                                           policy.b_in: all_Y_d_x,
                                           policy.c_in: slots,
                                           policy.b_prev_in: prev_act})
    probs = policy.session.run(policy.alignment_history, feed_dict={policy.a_in: X,
                                           policy.b_in: all_Y_d_x,
                                           policy.c_in: slots,
                                           policy.b_prev_in: prev_act})
    
    del tracker
#     print(np.shape(dial_vec))
    if bot is False:
        return (dial_vec[0,-2,:],
                no_skip_gate[0,-2,:],
                attn_embed[0,-2,:],
                rnn_embed[0,-2,:],
                probs[1][0,-2,:])
    else:
        return bot_vec
    
def make_prediction(story):
    dial, _,_,_,_ = embedding_from_tracker(story, bot=False)
    bot = embedding_from_tracker(story, bot=True)[0,-2,:,:]
    abs_vals = np.array([np.absolute(vec) for vec in bot])
    norms = np.apply_along_axis(np.linalg.norm, 1, bot)
    return domain.action_for_index(np.argmax(np.dot(bot, dial)/norms)).name()
#     print(dial.shape, bot.shape)

In [5]:
# print(domain.intents)
story_block = """* request_hotel
    - utter_ask_details
* explain
    - utter_explain_details_hotel
    - utter_ask_details
* chitchat
    - utter_chitchat
    - utter_ask_details
* inform{"enddate": "May 26th"}
    - slot{"enddate": "May 26th"}
    - utter_ask_startdate
* inform{"startdate": "next week"}
    - slot{"startdate": "next week"}
    - utter_ask_location
* explain
    - utter_explain_location_hotel
    - utter_ask_location
* inform{"location": "rome"}
    - slot{"location": "rome"}
    - utter_ask_price
* chitchat
    - utter_chitchat
    - utter_ask_price
* chitchat
    - utter_chitchat
    - utter_ask_price
* chitchat
    - utter_chitchat
    - utter_ask_price
* chitchat
    - utter_chitchat
    - utter_ask_price
* explain
    - utter_explain_price_hotel
    - utter_ask_price
* inform{"price": "expensive"}
    - utter_ask_people
* inform{"people": "4"}
    - utter_filled_slots
    - action_search_hotel
    - slot{"hotel": "hotel"}
    - utter_suggest_hotel
* chitchat
    - utter_chitchat
    - utter_suggest_hotel
* affirm
    - utter_happy"""
print(len(story_block.split('\n')))
cool_list = []
lines = story_block.split('\n')
for idx, line in enumerate(lines):
    if idx == len(lines)-1:
        continue
    if '*' in lines[idx+1] or 'slot{' in lines[idx+1]:# or 'slot{' in line:
        continue
    sub_story = '\n'.join(lines[:idx+1])
    (dial, not_skip, attn, rnn, probs
    ) = embedding_from_tracker("#generate story\n{}".format(sub_story))
    pred = make_prediction("#generate story\n{}".format(sub_story))
    cool_list.append((lines[idx+1], dial, pred, not_skip, attn[20:40], rnn, probs))

print(len(cool_list))
print(cool_list[0][4].shape)
#     break
# dial = embedding_from_tracker("#generate story\n{}".format(story_block)
# )
# print(dial.shape)
# print(len(story_block.split('\n')))

47
27
(20,)


In [6]:
cool_list[1]

('    - utter_explain_details_hotel',
 array([-0.02088561, -0.01536289,  0.03381712,  0.09675185,  0.10561556,
         0.04323883, -0.31790456,  0.34192157,  0.03062692,  0.1063322 ,
        -0.02378547, -0.0366603 ,  0.0616355 ,  0.02565157,  0.13443232,
        -0.21763241,  0.01394667,  0.2174925 , -0.0539252 ,  0.02394795],
       dtype=float32),
 'utter_explain_people_hotel',
 array([0.10599091], dtype=float32),
 array([-0.04794408,  0.05681676,  0.04003677, -0.03631898, -0.03891982,
         0.00771457,  0.02548517,  0.02850375, -0.02234503,  0.02146911,
         0.10654695,  0.0622694 , -0.06699082,  0.00337493, -0.05456778,
         0.00554225, -0.05081066, -0.00909461,  0.01054989, -0.02704157],
       dtype=float32),
 array([ 0.02705847, -0.07217965, -0.00621965,  0.13307084,  0.14453538,
         0.03552426, -0.34338972,  0.31341782,  0.05297195,  0.08486309,
        -0.13033243, -0.0989297 ,  0.12862632,  0.02227664,  0.18900008,
        -0.22317466,  0.06475734,  0.226587

In [7]:
from sklearn.decomposition import PCA

In [8]:
pca = PCA(n_components=2)
X = pca.fit_transform([ting[1] for ting in cool_list])

In [9]:
from bokeh.plotting import figure, show
import matplotlib.pyplot as plt

In [10]:
embed_dim = cool_list[0][1].shape[0]


In [11]:
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool, Arrow, OpenHead

hover = HoverTool(tooltips=[("true action","@desc"), ("prediction", "@pred"),
                            ("not_skip_gate", "@not_skip")],names=['circle'])
p = figure( tools=[hover])

for i in range(1, len(X)):
    p.add_layout(Arrow(end=OpenHead(line_color="firebrick", size=10, line_width=2),
                       x_start=X[i-1,0], y_start=X[i-1,1],
                       x_end=X[i,0], y_end=X[i,1], line_color='gray'))

source = ColumnDataSource(data=dict(
    x=X[:,0],
    y=X[:,1],
    desc=[ting[0] for ting in cool_list],
    pred = [ting[2] for ting in cool_list],
    not_skip = [ting[3] for ting in cool_list]
))
p.circle('x','y', source=source, size=10, name='circle')
# p.line(X[:,0], X[:,1])

X1 = pca.transform([ting[4] for ting in cool_list])
source = ColumnDataSource(data=dict(
    x=X1[:,0],
    y=X1[:,1],
    desc=[ting[0] for ting in cool_list],
    pred = [ting[2] for ting in cool_list],
    not_skip = [ting[3] for ting in cool_list]
))
p.circle('x','y', source=source, size=5, color='green', name='circle')

X2 = pca.transform([ting[5] for ting in cool_list])
source = ColumnDataSource(data=dict(
    x=X2[:,0],
    y=X2[:,1],
    desc=[ting[0] for ting in cool_list],
    pred = [ting[2] for ting in cool_list],
    not_skip = [ting[3] for ting in cool_list]
))
p.circle('x','y', source=source, size=5, color='red', name='circle')

x = pca.transform(np.zeros((1,embed_dim)))
p.circle(x[0,0], x[0,1], size=5, color='magenta')
p.circle(X[0,0], X[0,1], size=10, color='green')
p.circle(X[-1,0], X[-1,1], size=10, color='red')


GlyphRenderer(id='60de3838-017a-4e63-b503-b55206ce711a', ...)

In [12]:
show(p)

In [13]:
print(domain.index_for_action('utter_ask_price'))
print(domain.index_for_action('utter_ask_details'))
print(domain.index_for_action('utter_ask_location'))
print(domain.index_for_action('utter_ask_startdate'))
print(domain.index_for_action('utter_suggest_hotel'))
print(domain.index_for_action('action_search_hotel'))
print(domain.index_for_action('action_listen'))
print(domain.action_for_index(16))

5
27
4
6
11
41
0
UtterAction('utter_explain_location_hotel')


In [14]:
utter_price = embedding_from_tracker("#generate story\n{}".format(story_block), bot=True)[0,-1,5,:]
utter_price.shape

(20,)

In [15]:
print(utter_price.shape)
x = pca.transform(utter_price.reshape(1,-1))
p.circle(x[0][0], x[0][1], color='black')

(20,)


GlyphRenderer(id='4de4d927-7126-424c-b5b8-334e633e65f0', ...)

In [16]:
show(p)